In [ ]:
# Serialización del modelo final para despliegue

print("=== Proceso de Serialización del Modelo Final ===")

# Verificación de la estructura del directorio de modelos
print(f"Directorio de destino: {models_path}")
print(f"Directorio existe: {models_path.exists()}")

if not models_path.exists():
    models_path.mkdir(parents=True, exist_ok=True)
    print("Directorio de modelos creado exitosamente")

# Definición del nombre del archivo del modelo
model_filename = 'modelo_mantenimiento_predictivo.joblib'
model_path = models_path / model_filename

print(f"\nGuardando modelo final: {best_model_name}")
print(f"Ruta de destino: {model_path}")

try:
    # Serialización del pipeline completo
    joblib.dump(best_pipeline, model_path)
    
    # Verificación de la serialización
    file_size = model_path.stat().st_size / 1024  # Tamaño en KB
    print(f"Modelo guardado exitosamente")
    print(f"Tamaño del archivo: {file_size:.2f} KB")
    
except Exception as e:
    print(f"Error durante la serialización: {e}")
    raise

# Creación de archivo de metadatos del modelo
metadata = {
    'modelo_seleccionado': best_model_name,
    'fecha_entrenamiento': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S'),
    'metricas_rendimiento': {
        'auc_roc': float(detailed_metrics[best_model_name]['auc_roc']),
        'f1_score': float(detailed_metrics[best_model_name]['f1_score']),
        'precision_clase_pre_falla': float(detailed_metrics[best_model_name]['precision_class_1']),
        'recall_clase_pre_falla': float(detailed_metrics[best_model_name]['recall_class_1']),
        'tiempo_entrenamiento_segundos': float(detailed_metrics[best_model_name]['training_time'])
    },
    'datos_entrenamiento': {
        'tamaño_conjunto_entrenamiento': int(len(X_train)),
        'tamaño_conjunto_prueba': int(len(X_test)),
        'numero_caracteristicas': int(X_train.shape[1]),
        'distribucion_clases_entrenamiento': {
            'clase_0_normal': int(y_train.value_counts()[0]),
            'clase_1_pre_falla': int(y_train.value_counts()[1])
        }
    },
    'configuracion_pipeline': {
        'preprocesamiento': 'StandardScaler',
        'algoritmo': best_model_name,
        'parametros_modelo': str(best_pipeline.named_steps['classifier'].get_params())
    }
}

# Guardar metadatos en formato JSON
import json
metadata_path = models_path / 'modelo_metadatos.json'

with open(metadata_path, 'w', encoding='utf-8') as f:
    json.dump(metadata, f, indent=2, ensure_ascii=False)

print(f"\nMetadatos del modelo guardados en: {metadata_path}")

# Verificación de carga del modelo (prueba de integridad)
print("\n=== Verificación de Integridad del Modelo Serializado ===")

try:
    # Carga del modelo guardado
    loaded_pipeline = joblib.load(model_path)
    
    # Verificación de estructura
    print(f"Pipeline cargado exitosamente")
    print(f"Pasos del pipeline: {list(loaded_pipeline.named_steps.keys())}")
    
    # Prueba de predicción con una muestra
    if len(X_test) > 0:
        sample_prediction = loaded_pipeline.predict(X_test.iloc[:1])
        sample_probability = loaded_pipeline.predict_proba(X_test.iloc[:1])
        
        print(f"Prueba de predicción exitosa:")
        print(f"  Predicción: {sample_prediction[0]}")
        print(f"  Probabilidades: {sample_probability[0]}")
    
    # Verificación de equivalencia con el modelo original
    original_predictions = best_pipeline.predict(X_test.iloc[:10])
    loaded_predictions = loaded_pipeline.predict(X_test.iloc[:10])
    
    predictions_match = np.array_equal(original_predictions, loaded_predictions)
    print(f"  Equivalencia con modelo original: {predictions_match}")
    
    if predictions_match:
        print("✅ Verificación de integridad EXITOSA")
    else:
        print("❌ Verificación de integridad FALLIDA")
        
except Exception as e:
    print(f"Error durante la verificación: {e}")
    raise

print(f"\n=== Resumen del Modelo Final ===")
print(f"Modelo serializado: {best_model_name}")
print(f"Archivo del modelo: {model_filename}")
print(f"Tamaño del archivo: {file_size:.2f} KB")
print(f"Rendimiento (AUC-ROC): {detailed_metrics[best_model_name]['auc_roc']:.4f}")
print(f"Rendimiento (F1-Score): {detailed_metrics[best_model_name]['f1_score']:.4f}")
print(f"Recall clase pre-falla: {detailed_metrics[best_model_name]['recall_class_1']:.4f}")
print("\n✅ Proceso de entrenamiento y serialización completado exitosamente")
print("\nEl modelo está listo para ser utilizado en la fase de evaluación (Notebook 05)")

## Paso 7: Guardado del Modelo Final (Serialización)

### Importancia de la Serialización del Pipeline Completo

La serialización del modelo entrenado constituye un paso crítico en el flujo de trabajo de machine learning, especialmente para aplicaciones de mantenimiento predictivo que requieren despliegue en entornos operacionales. El enfoque implementado guarda el pipeline completo, no únicamente el algoritmo de clasificación, por las siguientes razones técnicas:

#### Ventajas de Guardar el Pipeline Completo:

1. **Consistencia de Preprocesamiento**: El pipeline encapsula tanto el StandardScaler como el clasificador, garantizando que las nuevas muestras reciban exactamente las mismas transformaciones aplicadas durante el entrenamiento.

2. **Eliminación de Discrepancias**: Previene errores de implementación que podrían surgir al aplicar manualmente las transformaciones de normalización en el entorno de producción.

3. **Portabilidad**: El pipeline serializado contiene toda la información necesaria para realizar predicciones, incluyendo los parámetros de normalización calculados sobre el conjunto de entrenamiento.

4. **Versionado**: Facilita el control de versiones del modelo completo, incluyendo tanto el preprocesamiento como los parámetros del algoritmo.

### Formato de Serialización

Se utiliza joblib para la serialización, que ofrece mayor eficiencia que pickle para objetos NumPy y scikit-learn, especialmente importante en modelos con gran cantidad de parámetros como Random Forest.

### Uso Futuro del Modelo Serializado

El pipeline guardado podrá ser cargado en futuros notebooks de evaluación o en sistemas de producción para realizar predicciones en tiempo real sobre nuevos datos operacionales del moto-compresor.

In [ ]:
# Análisis de importancia de características para el modelo seleccionado

print(f"=== Análisis de Interpretabilidad - {best_model_name} ===")

# Verificación de capacidad de análisis de importancia
classifier = best_pipeline.named_steps['classifier']

if hasattr(classifier, 'feature_importances_'):
    print(f"Analizando importancia de características para {best_model_name}...")
    
    # Extracción de importancias de características
    feature_importances = classifier.feature_importances_
    feature_names = X_train.columns
    
    # Creación de DataFrame con importancias
    importance_df = pd.DataFrame({
        'caracteristica': feature_names,
        'importancia': feature_importances
    }).sort_values('importancia', ascending=False)
    
    # Estadísticas de importancia
    print(f"\\nEstadísticas de importancia de características:")
    print(f"Total de características: {len(feature_importances)}")
    print(f"Importancia máxima: {feature_importances.max():.6f}")
    print(f"Importancia mínima: {feature_importances.min():.6f}")
    print(f"Importancia media: {feature_importances.mean():.6f}")
    
    # Análisis de concentración de importancia
    top_20_importance = importance_df.head(20)['importancia'].sum()
    total_importance = importance_df['importancia'].sum()
    concentration_ratio = top_20_importance / total_importance
    
    print(f"Concentración de importancia en top 20 características: {concentration_ratio:.2%}")
    
    # Visualización de las características más importantes
    plt.figure(figsize=(12, 8))
    
    # Top 20 características más importantes
    top_features = importance_df.head(20)
    
    # Gráfico de barras horizontales
    bars = plt.barh(range(len(top_features)), top_features['importancia'], 
                    color=plt.cm.viridis(np.linspace(0, 1, len(top_features))))
    
    # Configuración del gráfico
    plt.yticks(range(len(top_features)), top_features['caracteristica'])
    plt.xlabel('Importancia de Característica')
    plt.title(f'Top 20 Características Más Importantes - {best_model_name}', 
              fontsize=14, fontweight='bold')
    plt.gca().invert_yaxis()  # Invertir para mostrar la más importante arriba
    
    # Añadir valores de importancia en las barras
    for i, (bar, importance) in enumerate(zip(bars, top_features['importancia'])):
        plt.text(bar.get_width() + 0.0001, bar.get_y() + bar.get_height()/2, 
                 f'{importance:.4f}', ha='left', va='center', fontsize=8)
    
    plt.grid(True, alpha=0.3, axis='x')
    plt.tight_layout()
    plt.show()
    
    # Tabla detallada de características más importantes
    print(f"\\n=== Top 15 Características Más Importantes ===\")
    top_15 = importance_df.head(15).copy()
    top_15['importancia_porcentual'] = (top_15['importancia'] / total_importance * 100).round(2)
    top_15.index = range(1, len(top_15) + 1)
    
    print(top_15[['caracteristica', 'importancia', 'importancia_porcentual']].to_string())
    
    # Análisis de tipos de características importantes
    print(f"\\n=== Análisis de Tipos de Características Importantes ===\")
    
    # Categorización de características por tipo
    feature_types = {
        'moving_avg': 0, 'lag': 0, 'rolling': 0, 'shift': 0, 
        'std': 0, 'min': 0, 'max': 0, 'mean': 0, 'original': 0
    }
    
    for feature in top_15['caracteristica']:
        feature_lower = feature.lower()
        categorized = False
        
        for feature_type in feature_types.keys():
            if feature_type in feature_lower:
                feature_types[feature_type] += 1
                categorized = True
                break
        
        if not categorized:
            feature_types['original'] += 1
    
    # Mostrar distribución de tipos
    print("Distribución de tipos de características en Top 15:")
    for feature_type, count in feature_types.items():
        if count > 0:
            print(f"  {feature_type.replace('_', ' ').title()}: {count} características")

elif hasattr(classifier, 'coef_'):
    print(f"Analizando coeficientes para {best_model_name}...")
    
    # Para modelos lineales, usar valores absolutos de coeficientes
    coefficients = np.abs(classifier.coef_[0])
    feature_names = X_train.columns
    
    # Creación de DataFrame con coeficientes
    importance_df = pd.DataFrame({
        'caracteristica': feature_names,
        'coeficiente_abs': coefficients
    }).sort_values('coeficiente_abs', ascending=False)
    
    print(f"\\n=== Top 15 Características por Magnitud de Coeficiente ===\")
    print(importance_df.head(15).to_string(index=False))
    
    # Visualización similar para modelos lineales
    plt.figure(figsize=(12, 8))
    top_features = importance_df.head(20)
    
    plt.barh(range(len(top_features)), top_features['coeficiente_abs'])
    plt.yticks(range(len(top_features)), top_features['caracteristica'])
    plt.xlabel('Magnitud del Coeficiente (Valor Absoluto)')
    plt.title(f'Top 20 Características por Coeficiente - {best_model_name}', 
              fontsize=14, fontweight='bold')
    plt.gca().invert_yaxis()
    plt.grid(True, alpha=0.3, axis='x')
    plt.tight_layout()
    plt.show()

else:
    print(f"El modelo {best_model_name} no proporciona medidas de importancia de características interpretables.")
    print("Se recomienda utilizar métodos de interpretabilidad externos como SHAP para análisis detallado.")

print(f"\\nAnálisis de interpretabilidad completado para {best_model_name}")

In [ ]:
# Selección del mejor modelo basada en criterios múltiples

print("=== Proceso de Selección del Modelo Óptimo ===")

# Cálculo de métricas detalladas para cada modelo
detailed_metrics = {}
model_names = list(results.keys())

for model_name in model_names:
    y_pred = results[model_name]['predictions']
    
    # Cálculo del reporte de clasificación en formato de diccionario
    class_report = classification_report(y_test, y_pred, output_dict=True, zero_division=0)
    
    detailed_metrics[model_name] = {
        'auc_roc': results[model_name]['auc_roc'],
        'f1_score': results[model_name]['f1_score'],
        'precision_class_1': class_report['1']['precision'],
        'recall_class_1': class_report['1']['recall'],
        'training_time': results[model_name]['training_time']
    }

# Creación de DataFrame para análisis comparativo
metrics_df = pd.DataFrame(detailed_metrics).T
metrics_df = metrics_df.round(4)

print("Métricas detalladas por modelo:")
print(metrics_df)

# Implementación de sistema de puntuación ponderada para selección
print("\n=== Sistema de Puntuación para Selección de Modelo ===")

# Pesos asignados a cada métrica (ajustables según prioridades operacionales)
weights = {
    'auc_roc': 0.25,           # Capacidad general de discriminación
    'f1_score': 0.30,          # Balance precision-recall
    'recall_class_1': 0.35,    # Crítico: detección de pre-fallas
    'training_efficiency': 0.10 # Eficiencia computacional
}

print(f"Pesos asignados: {weights}")

# Normalización de métricas y cálculo de puntuación
scores = {}
max_training_time = max([detailed_metrics[name]['training_time'] for name in model_names])

for model_name in model_names:
    metrics = detailed_metrics[model_name]
    
    # Normalización de eficiencia computacional (invertida: menor tiempo = mejor)
    training_efficiency = 1 - (metrics['training_time'] / max_training_time)
    
    # Cálculo de puntuación ponderada
    score = (
        weights['auc_roc'] * metrics['auc_roc'] +
        weights['f1_score'] * metrics['f1_score'] +
        weights['recall_class_1'] * metrics['recall_class_1'] +
        weights['training_efficiency'] * training_efficiency
    )
    
    scores[model_name] = {
        'puntuacion_total': score,
        'training_efficiency': training_efficiency
    }

# Identificación del mejor modelo
best_model_name = max(scores.keys(), key=lambda x: scores[x]['puntuacion_total'])
best_pipeline = trained_pipelines[best_model_name]

print(f"\n=== Resultado de la Selección ===")
print(f"Modelo seleccionado: {best_model_name}")
print(f"Puntuación total: {scores[best_model_name]['puntuacion_total']:.4f}")

# Resumen de rendimiento del modelo seleccionado
best_metrics = detailed_metrics[best_model_name]
print(f"\nRendimiento del modelo seleccionado:")
print(f"  AUC-ROC: {best_metrics['auc_roc']:.4f}")
print(f"  F1-Score: {best_metrics['f1_score']:.4f}")
print(f"  Precision Clase Pre-Falla: {best_metrics['precision_class_1']:.4f}")
print(f"  Recall Clase Pre-Falla: {best_metrics['recall_class_1']:.4f}")
print(f"  Tiempo de Entrenamiento: {best_metrics['training_time']:.2f} segundos")

# Justificación de la selección
print(f"\n=== Justificación de la Selección ===")
print(f"El modelo {best_model_name} fue seleccionado basándose en:")
print(f"1. Alto recall para clase pre-falla: {best_metrics['recall_class_1']:.4f} (crítico para detectar fallas)")
print(f"2. F1-score balanceado: {best_metrics['f1_score']:.4f} (equilibrio precision-recall)")
print(f"3. Capacidad de discriminación: AUC-ROC {best_metrics['auc_roc']:.4f}")
print(f"4. Eficiencia computacional aceptable: {best_metrics['training_time']:.2f}s")

## Paso 6: Selección del Mejor Modelo y Análisis de Importancia

### Criterios de Selección del Modelo Óptimo

La selección del modelo óptimo para aplicaciones de mantenimiento predictivo requiere un análisis multidimensional que considere tanto el rendimiento predictivo como las restricciones operacionales. Los criterios de evaluación implementados priorizan:

#### Métricas Críticas para Mantenimiento Predictivo:

1. **Recall (Sensibilidad) para Clase Pre-Falla**: Capacidad del modelo para detectar correctamente las situaciones de pre-falla. En mantenimiento predictivo, los falsos negativos (fallas no detectadas) tienen consecuencias operacionales críticas.

2. **F1-Score**: Métrica que equilibra precisión y recall, especialmente relevante en contextos de desbalance de clases donde ambas métricas son importantes.

3. **AUC-ROC**: Capacidad general de discriminación entre clases, independiente del umbral de clasificación.

4. **Eficiencia Computacional**: Tiempo de entrenamiento y complejidad del modelo, factores determinantes para la implementación operacional.

### Justificación de la Selección

La selección se basa en una evaluación integral que prioriza el recall para la clase pre-falla sobre otras métricas, dado que el costo de no detectar una falla inminente es significativamente mayor que el de una falsa alarma en aplicaciones de mantenimiento industrial.

### Interpretabilidad del Modelo Seleccionado

En aplicaciones industriales críticas, la interpretabilidad del modelo es fundamental para:
- Validación por expertos en mantenimiento
- Identificación de sensores y variables críticas
- Optimización de estrategias de monitoreo
- Cumplimiento de requisitos regulatorios

Para modelos de tipo Random Forest, el análisis de importancia de características proporciona insights valiosos sobre qué variables operacionales son más predictivas de fallas inminentes.

In [ ]:
# Definición de algoritmos de clasificación para evaluación

print("=== Configuración de Algoritmos de Clasificación ===")

# Diccionario de modelos con configuraciones optimizadas para desbalance de clases
models = {
    'Logistic Regression': LogisticRegression(
        class_weight='balanced',  # Manejo automático de desbalance
        random_state=42,          # Reproducibilidad
        max_iter=1000,           # Suficientes iteraciones para convergencia
        solver='liblinear'        # Solver eficiente para datasets moderados
    ),
    
    'Random Forest': RandomForestClassifier(
        class_weight='balanced',  # Manejo automático de desbalance
        random_state=42,          # Reproducibilidad
        n_estimators=100,         # Balance entre rendimiento y velocidad
        max_depth=10,            # Prevención de overfitting
        min_samples_split=5,      # Criterio conservador para divisiones
        min_samples_leaf=2        # Hojas con múltiples muestras
    )
}

print(f"Algoritmos configurados: {list(models.keys())}")
print(f"Todos los modelos incluyen manejo de desbalance de clases")

# Contenedor para almacenar resultados de evaluación
results = {}
trained_pipelines = {}

print("\n" + "="*80)
print("INICIO DEL PROCESO DE ENTRENAMIENTO Y EVALUACIÓN")
print("="*80)

# Iteración a través de cada algoritmo
for model_name, model in models.items():
    print(f"\n{'='*20} {model_name} {'='*20}")
    
    # Creación del pipeline específico para este modelo
    pipeline = create_pipeline(model)
    
    print(f"Entrenando {model_name}...")
    
    # Entrenamiento del pipeline
    import time
    start_time = time.time()
    
    pipeline.fit(X_train, y_train)
    
    training_time = time.time() - start_time
    print(f"Tiempo de entrenamiento: {training_time:.2f} segundos")
    
    # Predicciones sobre conjunto de prueba
    y_pred = pipeline.predict(X_test)
    y_pred_proba = pipeline.predict_proba(X_test)[:, 1]  # Probabilidades clase positiva
    
    # Cálculo de métricas de evaluación
    auc_roc = roc_auc_score(y_test, y_pred_proba)
    f1 = f1_score(y_test, y_pred)
    
    # Almacenamiento de resultados
    results[model_name] = {
        'predictions': y_pred,
        'probabilities': y_pred_proba,
        'auc_roc': auc_roc,
        'f1_score': f1,
        'training_time': training_time
    }
    trained_pipelines[model_name] = pipeline
    
    print(f"AUC-ROC: {auc_roc:.4f}")
    print(f"F1-Score: {f1:.4f}")
    
    # Matriz de confusión visualizada
    plt.figure(figsize=(8, 6))
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['Normal', 'Pre-falla'],
                yticklabels=['Normal', 'Pre-falla'])
    plt.title(f'Matriz de Confusión - {model_name}')
    plt.ylabel('Valor Real')
    plt.xlabel('Predicción')
    plt.show()
    
    # Reporte de clasificación detallado
    print(f"\n--- Reporte de Clasificación - {model_name} ---")
    print(classification_report(y_test, y_pred, 
                              target_names=['Normal', 'Pre-falla'],
                              digits=4))

print("\n" + "="*80)
print("ENTRENAMIENTO Y EVALUACIÓN COMPLETADOS")
print("="*80)

## Paso 5: Entrenamiento y Evaluación de Modelos Base

### Estrategia de Selección de Algoritmos

Para establecer una línea base sólida de rendimiento, se implementarán múltiples algoritmos de clasificación que cubren diferentes aproximaciones metodológicas al problema de predicción de fallas. La selección de algoritmos se basa en:

#### Algoritmos Implementados:

1. **Regresión Logística**: 
   - Modelo lineal interpretable y computacionalmente eficiente
   - Proporciona probabilidades calibradas de clasificación
   - Establece una línea base sólida para comparación

2. **Random Forest Classifier**:
   - Ensemble de árboles de decisión con alta capacidad predictiva
   - Robusto ante overfitting y capaz de capturar interacciones complejas
   - Proporciona medidas de importancia de características

### Manejo del Desbalance de Clases

Dado el desbalance inherente en datos de mantenimiento predictivo, todos los algoritmos implementarán `class_weight='balanced'`. Esta configuración ajusta automáticamente los pesos de las clases de forma inversamente proporcional a su frecuencia, mejorando la capacidad del modelo para detectar la clase minoritaria (pre-falla).

### Métricas de Evaluación para Clasificación Desbalanceada

La evaluación se realizará utilizando métricas específicamente apropiadas para problemas de clasificación desbalanceada:

- **Matriz de Confusión**: Análisis detallado de verdaderos/falsos positivos y negativos
- **Precision, Recall y F1-Score**: Especialmente críticos para la clase minoritaria
- **AUC-ROC**: Capacidad de discriminación entre clases
- **Curva Precision-Recall**: Más informativa que ROC en casos de desbalance extremo

In [ ]:
# Creación del pipeline de modelado

print("=== Configuración del Pipeline de Modelado ===")

# Definición de la estructura base del pipeline
# El pipeline incluye dos etapas principales:
# 1. Preprocesamiento: StandardScaler para normalización de características
# 2. Clasificador: Será intercambiable según el algoritmo a evaluar

def create_pipeline(classifier):
    """
    Crea un pipeline de machine learning con preprocesamiento y clasificación.
    
    Args:
        classifier: Algoritmo de clasificación de scikit-learn
        
    Returns:
        Pipeline configurado con StandardScaler y el clasificador especificado
    """
    pipeline = Pipeline([
        ('scaler', StandardScaler()),  # Normalización de características
        ('classifier', classifier)      # Algoritmo de clasificación
    ])
    return pipeline

# Verificación de la estructura de características
print(f"Número de características de entrada: {X_train.shape[1]}")
print(f"Tipos de datos en características:")
print(X_train.dtypes.value_counts())

# Análisis estadístico básico de las características
print(f"\n=== Estadísticos de las Características (Conjunto de Entrenamiento) ===")
stats_summary = X_train.describe()
print(f"Rango de medias: {stats_summary.loc['mean'].min():.4f} a {stats_summary.loc['mean'].max():.4f}")
print(f"Rango de desviaciones estándar: {stats_summary.loc['std'].min():.4f} a {stats_summary.loc['std'].max():.4f}")
print(f"Rango de valores mínimos: {stats_summary.loc['min'].min():.4f} a {stats_summary.loc['min'].max():.4f}")
print(f"Rango de valores máximos: {stats_summary.loc['max'].min():.4f} a {stats_summary.loc['max'].max():.4f}")

# Demostración de la necesidad de normalización
scales_differ = (stats_summary.loc['std'].max() / stats_summary.loc['std'].min()) > 10
print(f"\n¿Las características tienen escalas significativamente diferentes? {scales_differ}")
print(f"Ratio de escala máxima/mínima: {stats_summary.loc['std'].max() / stats_summary.loc['std'].min():.1f}")

if scales_differ:
    print("La normalización es necesaria debido a las diferencias de escala entre características")
else:
    print("Las características tienen escalas similares, pero la normalización sigue siendo recomendable")

print("\nPipeline de modelado configurado exitosamente")
print("Estructura: StandardScaler → Clasificador")

## Paso 4: Creación del Pipeline de Modelado

### Arquitectura de Pipeline para Machine Learning

La implementación de un Pipeline de scikit-learn proporciona una arquitectura robusta y escalable para el proceso de modelado, encapsulando tanto el preprocesamiento como el algoritmo de clasificación en una unidad cohesiva. Esta aproximación ofrece múltiples ventajas operacionales:

#### Beneficios del Pipeline:

1. **Consistencia en el Preprocesamiento**: Garantiza que las mismas transformaciones aplicadas durante el entrenamiento se apliquen automáticamente durante la inferencia, eliminando discrepancias que podrían degradar el rendimiento del modelo.

2. **Prevención de Data Leakage**: El pipeline asegura que las estadísticas de normalización (media, desviación estándar) se calculen exclusivamente sobre el conjunto de entrenamiento y se apliquen posteriormente al conjunto de prueba.

3. **Reproducibilidad**: Encapsula todo el proceso de transformación y modelado en un objeto serializable, facilitando la reproducibilidad y el despliegue.

4. **Mantenibilidad**: Simplifica el código y reduce la posibilidad de errores al manejar múltiples pasos de procesamiento.

### Componentes del Pipeline:

1. **StandardScaler**: Normaliza las características para que tengan media cero y desviación estándar uno, requisito fundamental para algoritmos sensibles a la escala como la Regresión Logística.

2. **Clasificador**: Algoritmo de machine learning que será intercambiable para permitir la comparación de diferentes enfoques.

# Fase 4: Entrenamiento y Selección de Modelos de Clasificación

## Objetivo del Notebook

Este notebook constituye la cuarta fase del pipeline de mantenimiento predictivo para sistemas de moto-compresores. El objetivo principal es entrenar, evaluar y seleccionar un modelo de clasificación binaria capaz de predecir si ocurrirá una falla en los próximos 7 días, utilizando las características derivadas del proceso de ingeniería de características implementado en la fase anterior.

### Metodología de Entrenamiento

La metodología implementada se basa en principios fundamentales de machine learning para series temporales, priorizando:

1. **División Cronológica de Datos**: Implementación de una división temporal que respete la naturaleza secuencial de los datos operacionales, evitando la fuga de información (data leakage) que comprometería la validez del modelo.

2. **Manejo de Desbalance de Clases**: Aplicación de técnicas específicas para abordar la distribución desigual entre muestras de operación normal y muestras pre-falla, fundamental en aplicaciones de mantenimiento predictivo.

3. **Evaluación Robusta**: Utilización de métricas de rendimiento apropiadas para clasificación desbalanceada, que proporcionen una evaluación objetiva de la capacidad predictiva del modelo.

4. **Simplicidad Computacional**: Selección de algoritmos que mantengan un balance óptimo entre rendimiento predictivo y eficiencia computacional.

### Variable Objetivo Creada

La variable objetivo 'falla' ha sido creada basándose en el historial real de eventos de falla del compresor, donde:
- **Clase 0 (Normal)**: Operación normal del equipo
- **Clase 1 (Pre-falla)**: Muestras dentro de la ventana de 7 días previa a una falla documentada

### Librerías y Dependencias

El desarrollo requiere las siguientes librerías especializadas:
- **pandas, numpy**: Manipulación y procesamiento de datos
- **pathlib**: Gestión de rutas del sistema de archivos
- **joblib**: Serialización eficiente de modelos
- **matplotlib, seaborn**: Visualización de resultados y métricas
- **sklearn**: Algoritmos de machine learning, pipelines y métricas de evaluación

In [ ]:
# Importación de librerías fundamentales para manipulación de datos
import pandas as pd
import numpy as np
from pathlib import Path
import joblib
import warnings
warnings.filterwarnings('ignore')

# Librerías para visualización de resultados
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración de estilo para visualizaciones
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

# Importación de algoritmos de machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

# Importación de herramientas de pipeline y preprocesamiento
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Importación de métricas de evaluación para clasificación
from sklearn.metrics import (
    classification_report, 
    confusion_matrix, 
    roc_auc_score, 
    precision_recall_curve,
    roc_curve,
    auc,
    f1_score
)

# Definición de rutas del proyecto
data_processed_path = Path('./data/processed')
models_path = Path('./data/models')

# Creación de directorio de modelos si no existe
models_path.mkdir(exist_ok=True)

print("Configuración del entorno completada exitosamente")
print(f"Directorio de datos procesados: {data_processed_path}")
print(f"Directorio de modelos: {models_path}")

## Paso 2: Carga y Preparación de Datos

### Proceso de Carga de Dataset

En esta etapa se procede a cargar el dataset resultante del proceso de ingeniería de características desarrollado en la fase anterior, ahora enriquecido con la variable objetivo 'falla' creada a partir del historial real de eventos de mantenimiento. El archivo `featured_dataset_with_target.parquet` contiene:

- **Características originales de sensores**: Variables operacionales directas del moto-compresor
- **Características de ventanas móviles**: Estadísticos calculados sobre períodos temporales específicos
- **Características de lag temporal**: Variables retardadas que capturan dependencias temporales
- **Variable objetivo**: Indicador binario de proximidad a falla basado en eventos reales documentados

### Creación de la Variable Objetivo

La variable objetivo ha sido construida utilizando el historial de eventos de falla documentados en el archivo 'Historial C1 RGD.xlsx'. El proceso implementado:

1. **Extracción de Fechas**: Se procesaron automáticamente todas las fechas de eventos del historial
2. **Ventana de Pre-falla**: Se definió una ventana de 7 días previos a cada evento como período crítico
3. **Etiquetado**: Las muestras dentro de estas ventanas se etiquetaron como 'pre-falla' (1), el resto como 'normal' (0)

### Separación de Características y Variable Objetivo

La preparación de datos requiere la separación clara entre la matriz de características (X) y el vector de variable objetivo (y) para el entrenamiento supervisado del modelo de clasificación.

In [ ]:
# Carga del dataset con características de ingeniería y variable objetivo
dataset_path = data_processed_path / 'featured_dataset_with_target.parquet'

try:
    df = pd.read_parquet(dataset_path)
    print(f"Dataset cargado exitosamente desde: {dataset_path}")
    print(f"Dimensiones del dataset: {df.shape}")
    print(f"Memoria utilizada: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
except FileNotFoundError:
    print(f"Error: No se encontró el archivo {dataset_path}")
    print("Ejecutando creación de variable objetivo...")
    
    # Ejecutar script para crear variable objetivo
    import subprocess
    result = subprocess.run(['python', 'crear_variable_objetivo.py'], 
                          capture_output=True, text=True, cwd='.')
    print(result.stdout)
    if result.stderr:
        print(f"Errores: {result.stderr}")
    
    # Intentar cargar nuevamente
    df = pd.read_parquet(dataset_path)
    print(f"Dataset cargado exitosamente tras creación: {df.shape}")

# Verificación de la estructura del dataset
print("\n=== Información General del Dataset ===")
print(df.info())

# Análisis de la variable objetivo
print("\n=== Distribución de la Variable Objetivo 'falla' ===")
target_distribution = df['falla'].value_counts().sort_index()
print(target_distribution)
print(f"\nPorcentaje de muestras normales (0): {(target_distribution[0] / len(df) * 100):.2f}%")
print(f"Porcentaje de muestras pre-falla (1): {(target_distribution[1] / len(df) * 100):.2f}%")
print(f"Ratio de desbalance: {target_distribution[0] / target_distribution[1]:.1f}:1")

# Separación de características y variable objetivo
print("\n=== Preparación de Matrices de Entrenamiento ===")

# Definición de la matriz de características (X)
# Excluimos la columna 'falla' ya que es nuestra variable objetivo
feature_columns = [col for col in df.columns if col != 'falla']
X = df[feature_columns].copy()

# Definición del vector objetivo (y)
y = df['falla'].copy()

print(f"Matriz de características (X): {X.shape}")
print(f"Vector objetivo (y): {y.shape}")
print(f"Número de características disponibles: {len(feature_columns)}")

# Verificación de valores faltantes en características
missing_values = X.isnull().sum().sum()
print(f"Valores faltantes en características: {missing_values}")

if missing_values > 0:
    print("\nCaracterísticas con valores faltantes:")
    missing_by_column = X.isnull().sum()
    print(missing_by_column[missing_by_column > 0])

# Análisis de la distribución temporal de fallas
print(f"\n=== Análisis Temporal de la Variable Objetivo ===")
if isinstance(df.index, pd.DatetimeIndex):
    # Distribución por año
    df_temp = df.copy()
    df_temp['año'] = df_temp.index.year
    fallas_por_año = df_temp.groupby('año')['falla'].agg(['sum', 'count', 'mean'])
    fallas_por_año.columns = ['fallas', 'total_muestras', 'proporcion_fallas']
    print("Distribución de fallas por año:")
    print(fallas_por_año)
else:
    print("Dataset sin índice temporal - análisis temporal limitado")

## Paso 3: División Cronológica de Datos (Time-Based Split)

### Importancia Crítica de la División Temporal

La división de datos en series temporales requiere un enfoque metodológicamente diferente al utilizado en problemas de clasificación estándar. La función `train_test_split` de scikit-learn con `shuffle=True` es **fundamentalmente incorrecta** para datos de series temporales por las siguientes razones:

#### Problemas de la División Aleatoria:

1. **Fuga de Información (Data Leakage)**: Una división aleatoria permite que el modelo acceda a información futura durante el entrenamiento, creando una ventaja artificial que no existiría en un escenario de predicción real.

2. **Validación No Realista**: En aplicaciones industriales de mantenimiento predictivo, el modelo debe predecir eventos futuros basándose únicamente en datos históricos. Una división aleatoria no simula esta condición operacional.

3. **Sobreestimación del Rendimiento**: Los resultados obtenidos con división aleatoria tienden a sobrestimar significativamente la capacidad predictiva real del modelo.

#### Metodología de División Cronológica:

La división cronológica implementada respeta la naturaleza secuencial de los datos operacionales, utilizando un punto de corte temporal que separa:

- **Conjunto de Entrenamiento**: Datos históricos (80% inicial del dataset)
- **Conjunto de Prueba**: Datos más recientes (20% final del dataset)

Esta metodología simula fielmente el escenario operacional donde el modelo predice fallas futuras basándose únicamente en el historial de operación disponible hasta el momento de la predicción.

In [ ]:
# Implementación de división cronológica de datos

print("=== Implementación de División Cronológica ===")

# Definición del punto de corte temporal (80% para entrenamiento)
train_size = 0.8
split_index = int(len(df) * train_size)

print(f"Tamaño total del dataset: {len(df)} muestras")
print(f"Punto de corte temporal: índice {split_index}")
print(f"Proporción de entrenamiento: {train_size*100}%")
print(f"Proporción de prueba: {(1-train_size)*100}%")

# División cronológica de características
X_train = X.iloc[:split_index].copy()
X_test = X.iloc[split_index:].copy()

# División cronológica de variable objetivo
y_train = y.iloc[:split_index].copy()
y_test = y.iloc[split_index:].copy()

print(f"\n=== Dimensiones de los Conjuntos Resultantes ===")
print(f"X_train: {X_train.shape}")
print(f"X_test: {X_test.shape}")
print(f"y_train: {y_train.shape}")
print(f"y_test: {y_test.shape}")

# Análisis de distribución de clases en cada conjunto
print(f"\n=== Distribución de Clases por Conjunto ===")

train_distribution = y_train.value_counts().sort_index()
test_distribution = y_test.value_counts().sort_index()

print("Conjunto de Entrenamiento:")
print(f"  Clase 0 (normal): {train_distribution[0]} ({train_distribution[0]/len(y_train)*100:.2f}%)")
print(f"  Clase 1 (pre-falla): {train_distribution[1]} ({train_distribution[1]/len(y_train)*100:.2f}%)")
print(f"  Ratio de desbalance: {train_distribution[0]/train_distribution[1]:.1f}:1")

print("\nConjunto de Prueba:")
print(f"  Clase 0 (normal): {test_distribution[0]} ({test_distribution[0]/len(y_test)*100:.2f}%)")
print(f"  Clase 1 (pre-falla): {test_distribution[1]} ({test_distribution[1]/len(y_test)*100:.2f}%)")
print(f"  Ratio de desbalance: {test_distribution[0]/test_distribution[1]:.1f}:1")

# Visualización de la división cronológica
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Distribución temporal de la variable objetivo
ax1.plot(range(len(y)), y.values, alpha=0.7, linewidth=0.8)
ax1.axvline(x=split_index, color='red', linestyle='--', linewidth=2, label='Punto de División')
ax1.set_title('División Cronológica del Dataset')
ax1.set_xlabel('Índice Temporal')
ax1.set_ylabel('Variable Objetivo (falla)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Comparación de distribuciones de clases
sets = ['Entrenamiento', 'Prueba']
normal_counts = [train_distribution[0], test_distribution[0]]
failure_counts = [train_distribution[1], test_distribution[1]]

x = np.arange(len(sets))
width = 0.35

ax2.bar(x - width/2, normal_counts, width, label='Clase 0 (Normal)', alpha=0.8)
ax2.bar(x + width/2, failure_counts, width, label='Clase 1 (Pre-falla)', alpha=0.8)
ax2.set_title('Distribución de Clases por Conjunto')
ax2.set_xlabel('Conjunto de Datos')
ax2.set_ylabel('Número de Muestras')
ax2.set_xticks(x)
ax2.set_xticklabels(sets)
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nDivisión cronológica implementada correctamente")
print("El modelo será entrenado exclusivamente con datos históricos")
print("La evaluación se realizará sobre datos temporalmente posteriores")